# imports

In [1]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pickle
import qplib as qp
from qplib import log, na, nk, num



# log()

In [2]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)



log('trace: this is a trace message')
log('debug: this is a debug message')
log('info: this is an info message')
log('warning: this is a warning message')
log('error: this is an error message')



0,this is a trace message,,trace,2024-08-07 10:07:49.166771


1,this is a debug message,,debug,2024-08-07 10:07:50.449529


2,this is an info message,,info,2024-08-07 10:07:50.454223


3,this is a warning message,,Warning,2024-08-07 10:07:50.459660


4,this is an error message,,ERROR,2024-08-07 10:07:50.464675


# pd_query  






In [4]:

import numpy as np
import pandas as pd
import copy
import re
import qplib as qp

from IPython.display import display
from ipywidgets import widgets, interactive_output, HBox, VBox, fixed, Layout, interact_manual

from qplib.util import log
from qplib.types import qp_int, qp_float, qp_num, qp_bool, qp_datetime, qp_date, qp_na, qp_nk, qp_yn, qpDict
from qplib.pd_util import _check_df, _show_differences, _format_df, indexQpExtension, seriesQpExtension, dfQpExtension







if 'cards' not in globals():
    cards = pd.read_csv('data/cards.csv')
# cards.q('toughness ´r >2 & <5')
# cards.qi()


# df = qp.get_df()

# df.q(
#     r"""
#     # id ´r ?1  ´n test
#     # name ´m ~ x.upper()
#     # is any ´r is any

#     date of birth / age

#     """,
#     diff=None,
#     inplace=False,
#     verbosity=4,
#     )

# df.qi()



In [36]:

import cProfile, pstats

profiler = cProfile.Profile()
profiler.enable()

a =  cards.q(
    r"""
    toughness ´r >2 & <5
    """,
    diff=None,
    inplace=False,
    verbosity=3,
    )

profiler.disable()
stats = pstats.Stats(profiler).sort_stats('tottime')
stats.print_stats(10)



         100743 function calls (100583 primitive calls) in 0.135 seconds

   Ordered by: internal time
   List reduced from 458 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.061    0.015    0.061    0.015 {pandas._libs.lib.maybe_convert_numeric}
    83233    0.017    0.000    0.017    0.000 C:\Users\MartinVölkl-GouyaIns\AppData\Roaming\Python\Python312\site-packages\pandas\core\indexes\range.py:464(__iter__)
       11    0.014    0.001    0.053    0.005 C:\Users\MartinVölkl-GouyaIns\AppData\Local\Temp\ipykernel_26244\3984824421.py:604(match_symbol)
        3    0.003    0.001    0.003    0.001 {pandas._libs.lib.maybe_convert_objects}
        4    0.003    0.001    0.003    0.001 C:\Users\MartinVölkl-GouyaIns\AppData\Roaming\Python\Python312\site-packages\pandas\core\dtypes\cast.py:1544(construct_1d_object_array_from_listlike)
4250/4239    0.002    0.000    0.003    0.000 {built-in method builtins.isinstance}
      

In [2]:
cards.q(
    r"""
    toughness ´r >2 & <5
    """,
    diff=None,
    inplace=False,
    verbosity=3,
    )

,toughness
0,4
1,4
2,3
3,3
7,4
...,...
83212,4
83213,4
83216,4
83225,3


# qp.diff()

In [20]:

import pandas as pd
import numpy as np
import copy
import os
import datetime

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pd_util import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import qp_date, qp_na, qpDict




df_new, df_old = get_dfs()

_show_differences(df_new, df_old, show='new+', max_cols=200, max_rows=20, verbosity=3)




34,"adding column ""meta"" at position 0",df.format(),info,2024-07-17 12:26:24.573959


35,"adding column ""meta"" at position 0",df.format(),info,2024-07-17 12:26:24.587968


{'cols added': 1,
 'cols removed': 1,
 'rows added': 1,
 'rows removed': 1,
 'vals added': 1,
 'vals removed': 1,
 'vals changed': 1}

,meta,d,old: d,b,old: b,a,old: a
y,vals changed: 1,2.000000,,2.000000,,0.000000,2.000000
x2,added row,1.000000,,1.000000,,1.000000,
z,vals added: 1vals removed: 1,3.000000,,3.000000,None,nan,3.000000


# excel_diff()

In [5]:

import pandas as pd
import numpy as np
import copy
import os
import datetime

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pd_util import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import qp_date, qp_na, qpDict





# file_new = 'archive/stats_new.xlsx'
# file_old = 'archive/stats_old.xlsx'

# summary, results = excel_diff('archive/stats_new.xlsx', 'archive/stats_old.xlsx', to_excel=True)

# new, old = qp.get_dfs()
# display(new, old)
# _show_differences(new, old, mode='new+')

# a,b = excel_diff('archive/visit1_new.xlsx', 'archive/visit1_old.xlsx', file_diff='archive/visit1_diff.xlsx', index_col='ID', verbosity=3)

10,"adding column ""meta"" at position 0",df.format(),info,2024-08-06 14:48:03.102157


11,"adding column ""meta"" at position 0",df.format(),info,2024-08-06 14:48:03.110505


12,"differences saved to ""archive/visit1_diff.xlsx""",qp.excel_diff(),info,2024-08-06 14:48:04.808567


In [17]:
new, old = qp.get_dfs()
display(new, old)
_show_differences(new, old, mode='new+')

,d,b,a
y,2.0,2.0,0.0
x2,1.0,1.0,1.0
z,3.0,3.0,NaN


,a,b,c
x,1.0,1.0,1.0
y,2.0,2.0,2.0
z,3.0,None,3.0


12,"adding column ""meta"" at position 0",df.format(),info,2024-08-06 14:11:13.543954


13,"adding column ""meta"" at position 0",df.format(),info,2024-08-06 14:11:13.553222


{'cols added': 1,
 'cols removed': 1,
 'rows added': 1,
 'rows removed': 1,
 'vals added': 1,
 'vals removed': 1,
 'vals changed': 1}

,meta,d,old: d,b,old: b,a,old: a
y,vals changed: 1,2.000000,,2.000000,,0.000000,2.000000
x2,added row,1.000000,,1.000000,,1.000000,
z,vals added: 1vals removed: 1,3.000000,,3.000000,None,nan,3.000000


# pd_util

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import datetime
import qplib as qp

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.util import log, qpDict
from qplib.types import qp_date, qp_na






df = pd.DataFrame({
    'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
    'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
    'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
    'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
    'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
    'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
    'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
    'bp systole': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
    'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
    'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
    'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
    'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
    })

df_new, df_old = qp.get_dfs()






# types

In [ ]:
import pandas as pd
import numpy as np
import re


def qp_int(x, errors='coerce', na=np.nan):
    try:
        return int(float(x))  #float first to handle strings like '1.0'
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to integer.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns np.nan
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors

def qp_float(x, errors='coerce', na=np.nan):
    try:
        return float(x)
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to float.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns np.nan
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors
            
def qp_num(x, errors='coerce', na=np.nan):
    try:
        return pd.to_numeric(x)
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to numeric.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns np.nan
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors
            
def qp_bool(x, errors='coerce', na=None):
    if str(x).lower() in ['y', 'yes', 'true', '1', '1.0', 'positive', 'pos']:
        return True
    elif str(x).lower() in ['n', 'no', 'false', '0', '0.0', 'negative', 'neg']:
        return False
    else:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to boolean.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaN
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors


def qp_date(x, errors='coerce', na=pd.NaT):
    if isinstance(x, str):
        x = x.replace('_', '-')
    try:
        if re.match(r'\D*(1|2)\d\d\d', x):
            return pd.to_datetime(x, dayfirst=False).date()
        else:
            return pd.to_datetime(x, dayfirst=True).date()
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to date.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaT
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors
       
def qp_datetime(x, errors='coerce', na=pd.NaT):
    if isinstance(x, str):
        x = x.replace('_', '-')
    try:
        if re.match(r'\D*(1|2\d\d\d)', x):
            return pd.to_datetime(x, dayfirst=False)
        else:
            return pd.to_datetime(x, dayfirst=True)
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to datetime.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaT
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors


def qp_na(x, errors='ignore', na=None):
    possible_nas = [
        'not available', 'na', 'n/a', 'n.a', 'n.a.', 'na.', 'n.a',
        'not a number', 'nan',
        'null', 'nil',
        'none',
        '',
        ]
    
    if pd.isna(x) or str(x).lower() in possible_nas:
        return na
    else:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to "{na}".
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaN
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return None
            case _:
                return errors

def qp_nk(x, errors='ignore', nk='unknown'):
    possible_nks = [
        'unk', 'unknown', 'not known', 'not known.',
        'nk', 'n.k.', 'n.k', 'n/k',
        'not specified', 'not specified.',
        ]
    
    if str(x).lower() in possible_nks:
        return nk
    else:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to "{nk}".
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaN
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return None
            case _:
                return errors

def qp_yn(x, errors='coerce', yes='yes', no='no', na=None):
    if str(x).lower() in ['y', 'yes', 'true', '1', '1.0', 'positive', 'pos']:
        return yes
    elif str(x).lower() in ['n', 'no', 'false', '0', '0.0', 'negative', 'neg']:
        return no
    else:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to "{yes}" or "{no}".
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaN
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors

        




In [ ]:
import re

# Define the regex pattern to match a sequence of numbers allowing for different kinds of separators
pattern = r"\d+([,.\s_]\d+)*"

# Example string to match against
example_string = "123,456.789 0123\t456_789"

# Perform the search
matches = re.findall(pattern, example_string)

print(matches)

# "bashlike" wrappers

# temp

In [8]:
import qplib as qp
import pandas as pd

a, b = qp.get_dfs()

a.to_csv('a.csv', index=False)
b.to_csv('b.csv', index=False)

qp.diff(a,b)

8,"adding column ""meta"" at position 0",df.format(),info,2024-08-06 13:23:13.141031


9,"adding column ""meta"" at position 0",df.format(),info,2024-08-06 13:23:13.150751


{'cols added': 1,
 'cols removed': 1,
 'rows added': 1,
 'rows removed': 1,
 'vals added': 1,
 'vals removed': 1,
 'vals changed': 1}

,meta,d,b,a,c
y,vals changed: 1,2.000000,2.000000,0.000000,2.000000
x2,added row,1.000000,1.000000,1.000000,nan
z,vals added: 1vals removed: 1,3.000000,3.000000,nan,3.000000
x,removed row,nan,1.000000,1.000000,1.000000
